# How to onboard a new project to a cluster on Operate First
This guide explains how you can onboard a new project to one of the Operate First clusters. 

:::{seealso}

For more information about the Operate First see [Getting started with Operate First](references:operate-first-start)
:::

## Prerequisites
- There are no prerequisites.


## Outcomes

- [ ] A pull request against the `operate-first/apps` repository.

The PR enables Operate First to:

- [ ] Create a namespace on the desired Operate First cluster.
- [ ] Add desired users as namespace admins to the specified namespace.

## Introduction

All manifests for all the workloads owned by Operate First Ops team are maintained in the `operate-first/apps` repository following the [Kustomize best practices](https://kubernetes.io/docs/tasks/manage-kubernetes-objects/kustomization/).

The `cluster-scope` folder in this repo stores all privileged resources that are usually not allowed to be deployed by regular project admin and requires elevated access like cluster-admin role.

If you want to know more about the overall design please consult Operate First's [Architectural Decision Records (ADR) archive](https://www.operate-first.cloud/blueprints/blueprint/#architectural-decisions).


## Steps

### 1. Define important variables

In this guide we will use a couple of facts about the project. To make it easier to follow this guide, let's define these values beforehand.

In [2]:
import os
import json

# User variables
GITHUB_USERNAME = os.getenv("JUPYTERHUB_USER")  # If this notebook is executed within Jupyter Hub on Operate First, you can use the `JUPYTERHUB_USER` variables instead

# Namespace specific variables
NAMESPACE_NAME="demo-project"
NAMESPACE_DISPLAY_NAME="Demo Project Namespace"
TEAM_NAME="demo-team"

# Target cluster variables
TARGET_CLUSTER_NAME = "demo"
TARGET_CLUSTER_REGION = "emea"

NAMESPACE_ADMINS_LST = [GITHUB_USERNAME,] # list of LOWERCASE github usernames of the namespace admins

TARGET_CLUSTER=TARGET_CLUSTER_REGION+"/"+TARGET_CLUSTER_NAME

NAMESPACE_ADMINS=json.dumps([u.lower() for u in NAMESPACE_ADMINS_LST]).replace("\"", "\\\"")


### 2. Fork and clone the apps repository

Please fork/clone the [operate-first/apps](https://github.com/operate-first/apps) repository. We’ll be working within this repository only.

1. Go to [operate-first/apps](https://github.com/operate-first/apps).
2. Click on a fork button.
3. When a fork is created click on the code button and copy an address of your forked repository.
4. Run following command using copied address:

In [3]:
!git clone https://github.com/{GITHUB_USERNAME}/apps.git
%cd apps

Cloning into 'apps'...
remote: Enumerating objects: 12377, done.
remote: Counting objects: 100% (1658/1658), done.
remote: Compressing objects: 100% (877/877), done.
remote: Total 12377 (delta 743), reused 1561 (delta 686), pack-reused 10719
Receiving objects: 100% (12377/12377), 2.63 MiB | 4.05 MiB/s, done.
Resolving deltas: 100% (6003/6003), done.
/home/anand/Documents/4n4nd/hitchhikers-guide/pages/apps


### 3. Adding namespaces

For easier onboarding to ArgoCD later on, we prefer to follow a name pattern for all our namespaces. Please use your team name as a prefix to the namespace name like so: `$TEAM_NAME-example-project`.
####  Base resources


To create necessary resources use opfcli which creates the following folders with files:

- A namespace in `cluster-scope/base/core/namespaces/$NAMESPACE_NAME`.
- A blank user group for the `$TEAM_NAME` if it does not exist yet in the `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME`.
- An RBAC component for the project admin role `RoleBinding` in `cluster-scope/components/namespace-admin-rolebinding/$TEAM_NAME` and maps it to the newly added namespace.

In [4]:
!opfcli create-project {NAMESPACE_NAME} {TEAM_NAME} -d "{NAMESPACE_DISPLAY_NAME}"

INFO[0000] writing group definition to /home/anand/Documents/4n4nd/hitchhikers-guide/pages/apps/cluster-scope/base/user.openshift.io/groups/demo-team 
INFO[0000] writing rbac definition to /home/anand/Documents/4n4nd/hitchhikers-guide/pages/apps/cluster-scope/components/project-admin-rolebindings/demo-team 
INFO[0000] writing namespace definition to /home/anand/Documents/4n4nd/hitchhikers-guide/pages/apps/cluster-scope/base/core/namespaces/demo-project 


### 4. Adding namespace resources to the target cluster

The following cell will add `../../../../base/core/namespaces/$NAMESPACE_NAME` as a resource to the kusomization for your target cluster (`cluster-scope/overlays/prod/$TARGET_CLUSTER/kustomization.yaml`).




In [5]:
!cd cluster-scope/overlays/prod/{TARGET_CLUSTER}/ && kustomize edit add resource ../../../../base/core/namespaces/{NAMESPACE_NAME}

### 5. Populate your team
By executing the next cell, you will add the users listed in `$NAMESPACE_ADMINS_LST` to the OpenShift group `$TEAM_NAME` making them namespace admins for the above added namespace.

In [6]:
!yq e -i ".users = {NAMESPACE_ADMINS}" cluster-scope/base/user.openshift.io/groups/{TEAM_NAME}/group.yaml

## Finalize
Please stage your changes and send them as a PR against the [operate-first/apps](https://github.com/operate-first/apps) repository. 

:::{note}
Make sure that following files/ have been modified/added:
- [x] `cluster-scope/base/core/namespaces/$NAMESPACE_NAME/kustomization.yaml`
- [x] `cluster-scope/base/core/namespaces/$NAMESPACE_NAME/namespace.yaml`
- [x] `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME/group.yaml`
- [x] `cluster-scope/base/user.openshift.io/groups/$TEAM_NAME/kustomization.yaml`
- [x] `cluster-scope/components/namespace-admin-rolebinding/$TEAM_NAME/kustomization.yaml`
- [x] `cluster-scope/components/namespace-admin-rolebinding/$TEAM_NAME/rbac.yaml`
- [x] `cluster-scope/overlays/prod/$TARGET_CLUSTER/kustomization.yaml`
:::

Once the PR is merged, all the desired changes should be applied and the listed users should have admin access to the specified namespaces in the target cluster.

In [7]:
!git status
!git add .
!git commit -m "feat(onboarding): Add team {TEAM_NAME}"
!git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   cluster-scope/overlays/prod/emea/demo/kustomization.yaml

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cluster-scope/base/core/namespaces/demo-project/
	cluster-scope/base/user.openshift.io/groups/demo-team/
	cluster-scope/components/project-admin-rolebindings/demo-team/

no changes added to commit (use "git add" and/or "git commit -a")
[master bf5372e] feat(onboarding): Add team demo-team
 7 files changed, 41 insertions(+)
 create mode 100644 cluster-scope/base/core/namespaces/demo-project/kustomization.yaml
 create mode 100644 cluster-scope/base/core/namespaces/demo-project/namespace.yaml
 create mode 100644 cluster-scope/base/user.openshift.io/groups/demo-team/group.yaml
 create mode 100644 cluster-scope/ba